 #Detección de caras con webcam usando deepface

Deepface crea directorios para descargar los modelos, cuidado si tienes el disco bastante lleno. Es posible configurar la ruta, en mi caso uso  E:\RUNNERS_code\code\DeepFace, tras definir la ruta a través de la variable de entorno DEEPFACE_HOME

In [2]:
import cv2
from deepface import DeepFace

# documentation https://github.com/serengil/deepface/blob/master/deepface/modules/detection.py
# detector_backends deepface options: 'opencv', 'retinaface', 'mtcnn', 'ssd', 'dlib', 'mediapipe', 'yolov8', 'centerface' or 'skip'
detectors = ['opencv', 'mtcnn', 'retinaface', 'ssd']
detector_idx = 0

# Webcam connection
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
while True:
    # Read frame
    ret, frame = cap.read()
    if not ret:
        break

    try:
        # Face detection
        faces = DeepFace.extract_faces(frame, detector_backend=detectors[detector_idx])
        #print(faces)

        # Draw face bounding box and eyes locations
        for face in faces:
            x, y, w, h = face['facial_area']['x'], face['facial_area']['y'], face['facial_area']['w'], face['facial_area']['h']
            cv2.rectangle(frame, (x, y), (x + w, y + h), (200, 255, 200), 2)  # Dibuja el rectángulo
            if face['facial_area']['left_eye'] is not None:
                cv2.circle(frame, (face['facial_area']['left_eye'][0],face['facial_area']['left_eye'][1]), 3, (0, 255, 0), 2)
                cv2.circle(frame, (face['facial_area']['right_eye'][0],face['facial_area']['right_eye'][1]), 3, (0, 0, 255), 2)

    except Exception as e:
        print("Error:", e)

    # Mostrar el frame
    cv2.putText(frame, f"Detector: {detectors[detector_idx]}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    cv2.imshow("Video", frame)

    # Salir si se presiona Esc o cambia el detectr
    key = cv2.waitKey(1) & 0xFF
    if key == 27:  # Salir si se presiona 'q'
        break
    elif key == ord('d'):  # Cambiar detector si se presiona 'c'
        detector_idx = (detector_idx + 1) % len(detectors)
        print(f"Cambiado a detector: {detectors[detector_idx]}")

# Liberar la captura y cerrar ventanas
cap.release()
cv2.destroyAllWindows()


Error: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
Error: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
Error: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
Error: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
Error: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
Error: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
Error: Face could not be detected in numpy array.Please confirm that the picture is a face photo or 

Deepface y Retinaface únicamente

In [2]:
import cv2
import time
#from deepface import DeepFace
from retinaface import RetinaFace

# Webcam connection
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
# Check for other cameras
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print('Camera error')
            exit(0)
        else:
            print('Camera 0')
    else:
        print('Camera 1')
else:
    print('Camera 0')

#Set camera resolution
cap.set(3,640);
cap.set(4,480);

while True:
    # Get frame
    t = time.time()
    ret, frame = cap.read()

    #Deepface detecta caras y se queda con primera
    #face = DeepFace.detectFace(img_path = frame, 
    #    target_size = (224, 224), 
    #    detector_backend = backends[4]
    #)

    #Detectar múltiples (https://sefiks.com/2021/04/27/deep-face-detection-with-retinaface-in-python/)
    #Devuelve un dict
    faces = RetinaFace.detect_faces(frame);
    #Si hay detecciones
    if len(faces) > 0:
        #Dibuja contenedor y elementos faciales
        for idx in range(1,len(faces)+1):
            id = 'face_' + str(idx)
            #Muestra información de caras
            print(faces[id])
            #Accediendo a contenedor de la cara
            facial_area = faces[id]['facial_area']
            #Accediendo a datos de los elementos faciales
            landmarks = faces[id]['landmarks']
            #Debug
            #print(facial_area)
            #print(landmarks)
            
            #Contenedor
            cv2.rectangle(frame, (facial_area[2], facial_area[3])
            , (facial_area[0], facial_area[1]), (255, 255, 255), 1)

            #Elementos faciales
            cv2.circle(frame, (int(landmarks["left_eye"][0]),int(landmarks["left_eye"][1])), 3, (0, 0, 255), -1)
            cv2.circle(frame, (int(landmarks["right_eye"][0]),int(landmarks["right_eye"][1])), 3, (0, 0, 255), -1)
            cv2.circle(frame, (int(landmarks["nose"][0]),int(landmarks["nose"][1])), 3, (255, 0, 0), -1)
            cv2.circle(frame, (int(landmarks["mouth_left"][0]),int(landmarks["mouth_left"][1])), 3, (0, 255, 0), -1)
            cv2.circle(frame, (int(landmarks["mouth_right"][0]),int(landmarks["mouth_right"][1])), 3, (0, 255, 0), -1)
     
    #Muestra imagen
    
    cv2.imshow('Cam', frame)
    
    #Esperar por tecla o seguir
    tec = cv2.waitKey(40)
    if tec & tec == 27:  # Esc
        break
    
#Cerrar ventanas y cámara
cap.release()
cv2.destroyAllWindows()

Camera 0
{'score': np.float64(0.9993026256561279), 'facial_area': [np.int64(149), np.int64(0), np.int64(325), np.int64(242)], 'landmarks': {'right_eye': [np.float32(180.13342), np.float32(98.59678)], 'left_eye': [np.float32(256.30698), np.float32(92.60679)], 'nose': [np.float32(206.20424), np.float32(136.5342)], 'mouth_right': [np.float32(192.07889), np.float32(185.49884)], 'mouth_left': [np.float32(254.61784), np.float32(179.98872)]}}
{'score': np.float64(0.9993728995323181), 'facial_area': [np.int64(147), np.int64(0), np.int64(324), np.int64(242)], 'landmarks': {'right_eye': [np.float32(178.82051), np.float32(98.27505)], 'left_eye': [np.float32(255.21313), np.float32(93.02793)], 'nose': [np.float32(204.55644), np.float32(135.8625)], 'mouth_right': [np.float32(189.4495), np.float32(184.79091)], 'mouth_left': [np.float32(252.9751), np.float32(179.9392)]}}
{'score': np.float64(0.9980090260505676), 'facial_area': [np.int64(174), np.int64(2), np.int64(360), np.int64(268)], 'landmarks': {'